In [233]:
pip install xgboost lightgbm

Note: you may need to restart the kernel to use updated packages.


In [234]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [235]:
df=pd.read_csv('telco_churn.csv')

In [236]:
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        5043 non-null   int64  
 1   customerID        5043 non-null   object 
 2   gender            5043 non-null   object 
 3   SeniorCitizen     5043 non-null   object 
 4   Partner           5043 non-null   object 
 5   Dependents        5043 non-null   object 
 6   tenure            5043 non-null   int64  
 7   PhoneService      5043 non-null   object 
 8   MultipleLines     4774 non-null   object 
 9   InternetService   5043 non-null   object 
 10  OnlineSecurity    4392 non-null   object 
 11  OnlineBackup      4392 non-null   object 
 12  DeviceProtection  4392 non-null   object 
 13  TechSupport       4392 non-null   object 
 14  StreamingTV       4392 non-null   object 
 15  StreamingMovies   4392 non-null   object 
 16  Contract          5043 non-null   object 


,Unnamed: 0,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,7590-VHVEG,Female,False,True,False,1,False,NaN,DSL,...,False,False,False,False,Month-to-month,True,Electronic check,29.850000,29.850000381469727,False
1,1,5575-GNVDE,Male,False,False,False,34,True,False,DSL,...,True,False,False,False,One year,False,Mailed check,56.950001,1889.5,False
2,2,3668-QPYBK,Male,False,False,False,2,True,False,DSL,...,False,False,False,False,Month-to-month,True,Mailed check,53.849998,108.1500015258789,True
3,3,7795-CFOCW,Male,False,False,False,45,False,NaN,DSL,...,True,True,False,False,One year,False,Bank transfer (automatic),42.299999,1840.75,False
4,4,9237-HQITU,Female,False,False,False,2,True,False,Fiber optic,...,False,False,False,False,Month-to-month,True,Electronic check,70.699997,151.64999389648438,True
5,5,9305-CDSKC,Female,False,False,False,8,True,True,Fiber optic,...,True,False,True,True,Month-to-month,True,Electronic check,99.650002,820.5,True
6,6,1452-KIOVK,Male,False,False,True,22,True,True,Fiber optic,...,False,False,True,False,Month-to-month,True,Credit card (automatic),89.099998,1949.4000244140625,False
7,7,6713-OKOMC,Female,False,False,False,10,False,NaN,DSL,...,False,False,False,False,Month-to-month,False,Mailed check,29.750000,301.8999938964844,False
8,8,7892-POOKP,Female,False,True,False,28,True,True,Fiber optic,...,True,True,True,True,Month-to-month,True,Electronic check,104.800003,3046.050048828125,True
9,9,6388-TABGU,Male,False,False,True,62,True,False,DSL,...,False,False,False,False,One year,False,Bank transfer (automatic),56.150002,3487.949951171875,False


In [237]:
# Drop customerID and Unnamed: 0
df.drop(columns=['customerID', 'Unnamed: 0'], inplace=True)
df.shape


(5043, 20)

In [238]:
print('class distribution:',pd.Series(df['Churn']).value_counts())
df.head()

class distribution: Churn
False    2219
No       1487
True      780
Yes       556
Name: count, dtype: int64


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,False,True,False,1,False,NaN,DSL,False,True,False,False,False,False,Month-to-month,True,Electronic check,29.850000,29.850000381469727,False
1,Male,False,False,False,34,True,False,DSL,True,False,True,False,False,False,One year,False,Mailed check,56.950001,1889.5,False
2,Male,False,False,False,2,True,False,DSL,True,True,False,False,False,False,Month-to-month,True,Mailed check,53.849998,108.1500015258789,True
3,Male,False,False,False,45,False,NaN,DSL,True,False,True,True,False,False,One year,False,Bank transfer (automatic),42.299999,1840.75,False
4,Female,False,False,False,2,True,False,Fiber optic,False,False,False,False,False,False,Month-to-month,True,Electronic check,70.699997,151.64999389648438,True


In [239]:
import numpy as np

# Step 1: Clean the values
df['Churn'] = df['Churn'].astype(str).str.strip().str.lower()

# Step 2: Replace known variations with standardized values
df['Churn'] = df['Churn'].replace({
    'true': 'yes',
    '1': 'yes',
    '2': 'yes',
    'false': 'no',
    '0': 'no',
    'nan': np.nan,      
    '': np.nan,         
    'unknown': np.nan   
})

# Step 3: Drop rows where Churn is still nan
df = df.dropna(subset=['Churn'])

# Optional: Check final unique values (should only be 'yes' and 'no')
print(df['Churn'].unique())




['no' 'yes']


In [240]:
print(label['Churn'].classes_)  # Example output: ['no', 'yes']


['no' 'yes']


In [241]:
# handling missing values
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.fillna({'TotalCharges':df['TotalCharges'].median()}, inplace=True)


In [242]:

label={}
for col in df.columns:
    if df[col].dtype=='object':
        le=LabelEncoder()
        df[col]=le.fit_transform(df[col])
        label[col]=le
       

In [243]:
# Scale the numerical feature
sandard=StandardScaler()
numerical_features=['tenure','MonthlyCharges','TotalCharges']
df[numerical_features]=sandard.fit_transform(df[numerical_features])

In [244]:
#feature target and splitting
x=df.drop(columns=['Churn'])
y=df['Churn']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [245]:
print('class distribution',pd.Series(y_test.value_counts()))

class distribution Churn
0    759
1    250
Name: count, dtype: int64


In [246]:
smote=SMOTE(random_state=42)
x_train_resampled,y_train_resampled=smote.fit_resample(x_train,y_train)


In [247]:
print('class distribution',pd.Series(y_train_resampled.value_counts()))

class distribution Churn
1    2947
0    2947
Name: count, dtype: int64


In [248]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

In [249]:
#train random forest
rf=RandomForestClassifier(random_state=42)
rf.fit(x_train_resampled,y_train_resampled)
y_pred_rf=rf.predict(x_test)
roc_auc_score_rf=roc_auc_score(y_test,rf.predict_proba(x_test)[:,1])
print('Random Forest ROC AUC Score:', roc_auc_score_rf)
print('Random Forest Classification Report:\n', classification_report(y_test, y_pred_rf))

Random Forest ROC AUC Score: 0.8478392621870882
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.81      0.85       759
           1       0.56      0.71      0.63       250

    accuracy                           0.79      1009
   macro avg       0.73      0.76      0.74      1009
weighted avg       0.81      0.79      0.80      1009



In [250]:
#train xgboost
xgb=XGBClassifier(eval_metric='logloss',random_state=42)
xgb.fit(x_train_resampled,y_train_resampled)
y_pred_xgb=xgb.predict(x_test)
roc_auc_score_xgb=roc_auc_score(y_test,xgb.predict_proba(x_test)[:,1])
print('xgboost Forest ROC AUC Score:', roc_auc_score_xgb)
print('xgboost Forest Classification Report:\n', classification_report(y_test, y_pred_xgb))

xgboost Forest ROC AUC Score: 0.8303899868247694
xgboost Forest Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.82      0.85       759
           1       0.56      0.70      0.62       250

    accuracy                           0.79      1009
   macro avg       0.73      0.76      0.74      1009
weighted avg       0.81      0.79      0.80      1009



In [251]:
#train lightgbm
lgbm=LGBMClassifier(random_state=42)
lgbm.fit(x_train_resampled,y_train_resampled)
y_pred_lgbm=lgbm.predict(x_test)
roc_auc_score_lgbm=roc_auc_score(y_test,lgbm.predict_proba(x_test)[:,1])
print('LightGBM ROC AUC Score:', roc_auc_score_lgbm)
print('LightGBM Classification Report:\n', classification_report(y_test, y_pred_lgbm))

[LightGBM] [Info] Number of positive: 2947, number of negative: 2947
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001293 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 839
[LightGBM] [Info] Number of data points in the train set: 5894, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
LightGBM ROC AUC Score: 0.846052700922266
LightGBM Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.80      0.85       759
           1       0.54      0.73      0.62       250

    accuracy                           0.78      1009
   macro avg       0.72      0.77      0.74      1009
weighted avg       0.81      0.78      0.79      1009



In [252]:
#roc auc comparison
print('ROC AUC Scores:')
print('Random Forest:', roc_auc_score_rf)
print('XGBoost:', roc_auc_score_xgb)
print('LightGBM:', roc_auc_score_lgbm)

ROC AUC Scores:
Random Forest: 0.8478392621870882
XGBoost: 0.8303899868247694
LightGBM: 0.846052700922266
